# Solving SAT problem with Grover's algorithm

The **"Solving SAT problem with Grover's algorithm"** quantum kata is a series of exercises designed
to get you comfortable with using Grover's algorithm to solve realistic problems
using boolean satisfiability problems (SAT) as an example.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Oracles for SAT problems

The most interesting part of learning Grover's algorithm is solving realistic problems.
This means using oracles which express an actual problem and not simply hard-code a known solution.
In this section we'll learn how to express boolean satisfiability problems as quantum oracles.

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [ ]:
%kata T11_Oracle_And_Test 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [ ]:
%kata T12_Oracle_Or_Test 

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [ ]:
%kata T13_Oracle_Xor_Test 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

> This oracle marks two states similar to the state explored in task 1.2 of the GroversAlgorithm kata: 
$|10101...\rangle$ and $|01010...\rangle$.  
It is possible (and quite straightforward) to implement this oracle based on this observation; 
however, for the purposes of learning to write oracles to solve SAT problems we recommend using the representation above.

<details>
  <summary>Need a hint? Click here </summary>
  Remember that you can use operations defined in previous tasks.
</details>

In [ ]:
%kata T14_Oracle_AlternatingBits_Test 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{ik},\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[[(0, true), (1, false)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [ ]:
%kata T15_Oracle_SATClause_Test 

operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    // ...
}

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$)

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [ ]:
%kata T16_Oracle_SAT_Test 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    // ...
}

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [ ]:
%kata T21_Oracle_Exactly1One_Test 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // ...
}

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)

  2. a qubit in an arbitrary state $|y\rangle$ (target qubit)

  3. a 2-dimensional array of tuples `problem` which describes the 2-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in tasks 1.6 and 1.7;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true which are solutions for the general SAT problem.

In [ ]:
%kata T22_Oracle_Exactly1SAT_Test 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    // ...
}

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

> If you want to learn the Grover's algorithm itself, try doing [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) first.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<br/>
<details>
  <summary>Need some help? Click here </summary>
Experiment with SAT instances with different number of solutions and the number of algorithm iterations 
to see how the probability of the algorithm finding the correct answer changes depending on these two factors.

For example, 
* the AND oracle from task 1.1 has exactly one solution,
* the alternating bits oracle from task 1.4 has exactly two solutions,
* the OR oracle from task 1.2 for 2 qubits has exactly 3 solutions, and so on.
</details>

In [ ]:
operation Run_GroversSearch_Algorithm () : Unit {
    // ...
}

In [ ]:
%simulate Run_GroversSearch_Algorithm

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. the number of qubits N,

  2. a marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

> Note that the similar task in the GroversAlgorithm kata required you to implement Grover's algorithm 
in a way that would be robust to accidental failures, but you knew the optimal number of iterations 
(the number that minimized the probability of such failure).
> 
> In this task you also need to make your implementation robust to not knowing the optimal number of iterations.

In [ ]:
%kata T32_UniversalGroversAlgorithm_Test 

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    // ...
    return new Bool[N];
}